In [3]:
from google.colab import files
uploaded = files.upload()

Saving cidades_cost.xls to cidades_cost.xls


In [4]:
pip install gekko

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
from re import X

# -*- coding: utf-8 -*-
"""
TRABALHO 1 - P108
Joao Victor de O. G Ribeiro - GES 106
"""

import numpy as np
from xlrd import open_workbook
from gekko import GEKKO

m = 106

def somar_algarismos_matricula(numero):
    soma = 0
    for algarismo in str(numero):
        soma += int(algarismo)
    return soma

#a = (resto da divisão do número de matrícula 'm' por 5) + 1
a = (m%5)+1

#b = soma dos algarismos do número de matrícula
b = somar_algarismos_matricula(int(m))

#Peso dos arcos:
arcos = {
  'A': a+b,
  'B': a,
  'C': b,
  'D': a+5,
  'E': b-2,
  'F':a+3,
  'G': b-3,
  'H': a+4,
  'I': b-1,
  'J': a,
  'K': a+b,
  'L': b-2,
  'M': a+4,
  'N': b-1,
  'O': b,
  'P': a+5,
  'Q': b-3,
  'R': a+1,
  'S': a+3,
  'T': b-2,
  'U': b-1
}

#Read input data from spreadsheet.
wb = open_workbook('cidades_cost.xls', ignore_workbook_corruption=True)

for s in wb.sheets():
    number_of_nodes = 0
    for col in range(1,s.ncols):
        if (s.cell(0,col).value)!='':
            number_of_nodes = number_of_nodes + 1

#Create two dimensional matrix.
connection_cost_matrix = np.zeros((number_of_nodes,number_of_nodes), dtype=str)

for row in range(s.nrows):
    for col in range(s.ncols):
        if row > 0:
            value  = (s.cell(row,col).value)
            connection_cost_matrix[row-1,col-1] = value

print('ATIVIDADE 1 - Arvore geradora minima','\n')
print ('MATRIZ DE CUSTO DA ATIVIDADE 1')
print (connection_cost_matrix, '\n')

for i in range(len(connection_cost_matrix)):
    for j in range(len(connection_cost_matrix[i])):
        if connection_cost_matrix[i][j] in arcos:
            connection_cost_matrix[i][j] = int(arcos[connection_cost_matrix[i][j]])

# Convertendo a matriz de strings para inteiros
connection_cost_matrix_int = np.zeros((number_of_nodes, number_of_nodes), dtype=int)

for row in range(number_of_nodes):
    for col in range(number_of_nodes):
        connection_cost_matrix_int[row, col] = int(connection_cost_matrix[row, col])

# Imprimindo a matriz de custo convertida
print ('MATRIZ DE CUSTO COM OS VALORES REAIS DE ACORDO COM MINHA MATRICULA')
print(connection_cost_matrix_int, '\n')


#Excluding 0 in cost matrix and convert it to list.
connection_cost_matrix_int[connection_cost_matrix_int == 0] = 10*connection_cost_matrix_int.max()
cost = connection_cost_matrix_int.tolist()

#Initialize lists for iteractions.
not_connected = []
connected = []

#Start with all nodes not connected
for i in range(number_of_nodes):
    not_connected.append(i)

#First iteraction - start with first node
connected.append(min(not_connected)) #Include first not in connected.
not_connected.pop(not_connected.index(min(not_connected))) #Exclude first not
                                                           #from not_connected.

#Other iteractions
edge_from =[]
edge_to = []

total_cost = 0

while not_connected != []:

    temp_min = 10*connection_cost_matrix_int.max() #Stablish a big value for starting.

    for i in connected:
        for j in not_connected:
            if cost[i][j] < temp_min:
                temp_min = cost[i][j]
                from_node = i
                to_node = j

    total_cost = total_cost + temp_min
    connected.append(to_node)
    not_connected.pop(not_connected.index(to_node))

    edge_from.append(from_node)
    edge_to.append(to_node)

#Print Arcs
print ('RESULTADO:', '\n')
print ('Custo total de implantação da rede:', total_cost, 'Milhoes de reais', '\n')

print ('Arco/Cidade:')
for i in range(0,len(edge_from)):
    print ("Arco",i+1,":",'Cidade',edge_from[i]+1,"--->",'Cidade',edge_to[i]+1)

print('#########################################################################################################################','\n')
print('ATIVIDADE 2 - Caminho minimo', '\n')

z = GEKKO()
x12, x13, x14, x25, x26, x35, x36, x37, x46, x47, x58, x68, x78 = z.Array(z.Var,13,integer=True,lb=0)
z.Minimize(x12*arcos['U'] + x13*arcos['T'] + x14*arcos['S'] + x25*arcos['P'] + x26*arcos['Q'] + x35*arcos['R'] + x36*arcos['K'] + x37*arcos['L'] + x46*arcos['M'] + x47*arcos['N'] + x58*arcos['B'] + x68*arcos['C'] + x78*arcos['D'])
z.Equations([
    1 == x12+x13+x14,
    x12 == x26 + x25,
    x14 == x47 + x46,
    x13 == x35 + x36 + x37,
    x25 + x35 == x58,
    x26 + x36 + x46 == x58,
    x37 + x47 == x78
])

z.options.SOLVER = 1
z.solve()
print('Distancia total do caminho: ', z.options.OBJFCNVAL, 'km')

caminhos = {'1 -> 2': x12.value[0],
            '1 -> 3': x13.value[0],
            '1 -> 4': x14.value[0],
            '1 -> 5': x25.value[0],
            '2 -> 6': x26.value[0],
            '3 -> 5': x35.value[0],
            '3 -> 6': x36.value[0],
            '3 -> 7': x37.value[0],
            '4 -> 6': x46.value[0],
            '4 -> 7': x47.value[0],
            '5 -> 8': x58.value[0],
            '6 -> 8': x68.value[0],
            '7 -> 8': x78.value[0]
            }

print('CAMINHO:')
for caminho, valor in caminhos.items():
    if valor == 1.0:
        print(caminho)


print('#########################################################################################################################','\n')
print('ATIVIDADE 3 - Fluxo maximo', '\n')

g = GEKKO()
x14, x24, x25, x26, x35, x45, x46, x47, x56, x58, x67, x68 = g.Array(g.Var,12,integer=True,lb=0)
g.Maximize(x47 + x67 + x68 + x58) #Maximizing output.
g.Equations([ x14 + x24 == x46 + x47,
              x25 + x35 == x56 + x58,
              x26 + x46 + x56 == x67 + x68,
              x14<=arcos['A'],
              x24<=arcos['B'],
              x25<=arcos['D'],
              x26<=arcos['C'],
              x35<=arcos['E'],
              x45<=arcos['H'],
              x46<=arcos['G'],
              x47<=arcos['F'],
              x56<=arcos['I'],
              x58<=arcos['J'],
              x67<=arcos['K'],
              x68<=arcos['L'] ])
g.options.SOLVER = 1
g.solve()
print('Fluxo maximo: ', -g.options.OBJFCNVAL, 'm3/s')

print('x14: ', x14.value[0])
print('x24: ', x24.value[0])
print('x25: ', x25.value[0])
print('x26: ', x26.value[0])
print('x35: ', x35.value[0])
print('x45: ', x45.value[0])
print('x46: ', x46.value[0])
print('x47: ', x47.value[0])
print('x56: ', x56.value[0])
print('x58: ', x58.value[0])
print('x67: ', x67.value[0])
print('x68: ', x68.value[0])


ATIVIDADE 1 - Arvore geradora minima 

MATRIZ DE CUSTO DA ATIVIDADE 1
[['0' 'A' 'B' 'C' '0' '0' '0' '0']
 ['A' '0' '0' 'D' 'F' '0' '0' '0']
 ['B' '0' '0' 'E' '0' 'J' '0' '0']
 ['C' 'D' 'E' '0' 'G' 'I' 'H' '0']
 ['0' 'F' '0' 'G' '0' '0' 'L' 'K']
 ['0' '0' 'J' 'I' '0' '0' 'N' 'O']
 ['0' '0' '0' 'H' 'L' 'N' '0' 'M']
 ['0' '0' '0' '0' 'K' 'O' 'M' '0']] 

MATRIZ DE CUSTO COM OS VALORES REAIS DE ACORDO COM MINHA MATRICULA
[[0 9 2 7 0 0 0 0]
 [9 0 0 7 5 0 0 0]
 [2 0 0 5 0 2 0 0]
 [7 7 5 0 4 6 6 0]
 [0 5 0 4 0 0 5 9]
 [0 0 2 6 0 0 6 7]
 [0 0 0 6 5 6 0 6]
 [0 0 0 0 9 7 6 0]] 

RESULTADO: 

Custo total de implantação da rede: 29 Milhoes de reais 

Arco/Cidade:
Arco 1 : Cidade 1 ---> Cidade 3
Arco 2 : Cidade 3 ---> Cidade 6
Arco 3 : Cidade 3 ---> Cidade 4
Arco 4 : Cidade 4 ---> Cidade 5
Arco 5 : Cidade 5 ---> Cidade 2
Arco 6 : Cidade 5 ---> Cidade 7
Arco 7 : Cidade 7 ---> Cidade 8
#####################################################################################################################